# Dijkstra's Algorithm for Single-Sourced Shortest Path
> Graph Data Format
>> Node = 0~N-1  
>> Edge = (start_node, end_node, weight)

In [16]:
from typing import Tuple, List
import heapq

#Min-Priority Queue for Dijkstra's algorithm
class PriorityQueue:
    def __init__(self):
        self._queue = []
        self._entry_finder = {} #key : task, value : entry=[priority, self._counter, task]
        self._REMOVED = '<removed-task>'
        self._counter = 0

    def add_task(self, task, priority=0):
        ## remove the origin one
        if task in self._entry_finder:
            self.remove_task(task)
        entry = [priority, self._counter, task] #counter는 priority가 같을 경우에 순서를 비교하기 위한 용도
        self._counter += 1
        self._entry_finder[task] = entry
        heapq.heappush(self._queue, entry)

    def remove_task(self, task):
        entry = self._entry_finder.pop(task)
        entry[-1] = self._REMOVED #heap 내에 있는 entry도 수정이 됨(alias이기 때문)

    def pop_task(self):
        while self._queue:
            priority, count, task = heapq.heappop(self._queue)
            if task is not self._REMOVED:
                del self._entry_finder[task]
                return task
        raise KeyError('pop from an empty priority queue')
    
class Graph :
    def __init__(self, N:int, M:int, edge : List[Tuple[int, int, int]]) :
        self.num_vertex=N # vertices correspond to 0~N-1
        self.num_edge=M
        self.edge=edge #List[Tuple[int, int, int]]
        self.edge_dict=dict() #key:start node, values : (end node, weight)
        self.dist=[] #to record distance
        self.predecessor=[] #to record predecessor in the path

    def initialize_single_source(self, source:int) :
        self.dist=[float('inf')]*self.num_vertex #to record distance
        self.predecessor=[None]*self.num_vertex #to record predecessor in the path
        self.dist[source]=0#source point distance=0

    #to make adjacent list for algorithm
    def form_edge_dict(self) :
        self.edge_dict=dict()
        for i in range(self.num_vertex) :
            self.edge_dict[i]=[]
        for start, end, weight in self.edge :
            self.edge_dict[start].append((end, weight))

    def relax(self, u:int, v:int, weight_from_u_to_v:int) :
        if self.dist[v]>self.dist[u]+weight_from_u_to_v :
            self.dist[v]=self.dist[u]+weight_from_u_to_v
            self.predecessor[v]=u
            return True

    def print_path(self, start:int, end:int) :
        if start==end :
            print(f'({start})', end="")
        elif self.predecessor[end]==None :
            print(f'(-1) No path.', end="")
        else :
            self.print_path(start, self.predecessor[end])
            print(f'->({end})', end="")        


    def dijkstra_alg(self, source:int) :
        self.form_edge_dict()
        self.initialize_single_source(source)
        s=set()
        q=PriorityQueue()
        for i in range(self.num_vertex) :
            q.add_task(i, self.dist[i])
        print(q._queue)
        while len(s)<self.num_vertex :
            start=q.pop_task()
            print(f"{start} popped")
            s.add(start)
            for (end, weight) in self.edge_dict[start] :
                flag=self.relax(start, end, weight)
                if flag==True :
                    q.add_task(end, self.dist[end])
        for i in range(self.num_vertex) :
            print(f'({source})->({i}) distance=({self.dist[i]})\t',end="")
            self.print_path(source, i)
            print()


    def bellman_ford_alg(self, source:int) :
        self.initialize_single_source(source)
        for i in range(self.num_vertex-1) :
            for (start, end, weight) in self.edge :
                self.relax(start, end, weight)
        for (start, end, weight) in self.edge :
            if self.dist[end] > self.dist[start]+weight :
                return -1
        # for i in range(self.num_vertex) :
        #     print(self.dist[i])
        for i in range(self.num_vertex) :
            print(f'({source})->({i}) distance=({self.dist[i]})\t',end="")
            self.print_path(source, i)
            print()



#input은 index 시작이 1부터임
input="""5 6
1
5 1 1
1 2 2
1 3 3
2 3 4
2 4 5
3 4 6"""
input="""5 10
1
1 2 10
1 4 5
2 4 2
4 2 3
2 3 1
4 3 9
3 5 4
5 3 6
4 5 2
5 1 7"""
lines=input.splitlines()
n, m = map(int, lines[0].split())
s=int(lines[1])
s-=1
edge=[]
for i in range(2, m+1, 1) :
    a, b, c = map(int, lines[i].split())
    edge.append((a-1, b-1, c)) # 0-indexed로 바꾸어 다뤄주기
graph=Graph(n, m, edge)
graph.dijkstra_alg(s)

graph=Graph(n, m, edge)
graph.bellman_ford_alg(s)

# #py파일용
# if __name__ == "__main__":
#     n, m = map(int, input().split())
#     edge = []
    
#     for _ in range(m):
#         # start, end, weight = map(int, input().split())
#         # edge.append((start,end,weight))
#         edge.append((map(int, input().split())))
    
#     graph=Graph(n, m, edge)
#     graph.bellman_ford_alg(1)


[[0, 0, 0], [inf, 1, 1], [inf, 2, 2], [inf, 3, 3], [inf, 4, 4]]
0 popped
3 popped
4 popped
1 popped
2 popped
(0)->(0) distance=(0)	(0)
(0)->(1) distance=(8)	(0)->(3)->(1)
(0)->(2) distance=(9)	(0)->(3)->(1)->(2)
(0)->(3) distance=(5)	(0)->(3)
(0)->(4) distance=(7)	(0)->(3)->(4)
(0)->(0) distance=(0)	(0)
(0)->(1) distance=(8)	(0)->(3)->(1)
(0)->(2) distance=(9)	(0)->(3)->(1)->(2)
(0)->(3) distance=(5)	(0)->(3)
(0)->(4) distance=(7)	(0)->(3)->(4)


In [7]:
input="""3 4
1 2 4
1 3 3
2 3 -1
3 1 -2
"""
lines=input.splitlines()
print(lines)
n, m = map(int, lines[0].split())
print(n, m)


a, b, c = map(int, lines[4].split())
print(a)
print(b)
print(c)


['3 4', '1 2 4', '1 3 3', '2 3 -1', '3 1 -2']
3 4
3
1
-2
